In [3]:
320*60

19200

In [4]:
import numpy as np

from esc_utils import INPUT_LENGTH

import sys
sys.path.append("/workspace/jgusak/scatnet_python")
from scatnet_python import *


from IPython.display import Audio

import matplotlib.pylab as plt
%matplotlib inline

import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
import esc_utils as U

def create_dataset(npz_path, split = 4, fs = 16000,
                   augment_factor = 60,  strong = True):
# tfrecord_pathes = pathes for train, val tfrecords    
    with np.load(npz_path) as dataset:

        train_sounds, train_labels = [], []
        val_sounds, val_labels = [], []

        for i, fold in enumerate(dataset.files):
            sounds = dataset[fold].item()['sounds']
            labels = dataset[fold].item()['labels']
            
            
            # we'll add to dataset only samples with length >= 40000
#             idxs = list(filter(lambda i: len(sounds[i]) >= 40000,
#                                range(len(sounds))))
#             sounds = list(np.array(sounds)[idxs])
#             labels = list(np.array(labels)[idxs])
            
            print('Preprocessing sounds...')
            sounds = [U.preprocess_sound(sound) for sound in sounds]
#             print(len(sounds), len(labels))            
                    
            if i  == split:
                val_sounds.extend(sounds)
                val_labels.extend(labels)
            else:
                
                print('Augmenting data...')
                if augment_factor:
                    augmented_sounds, augmented_labels = [], []
                
                    for sound, label in zip(sounds, labels):
                        augmented_sounds.extend([U.augment_sound(sound, strong = strong)
                                                 for _ in range(augment_factor)])
                        augmented_labels.extend([label]*augment_factor)

                    sounds.extend(augmented_sounds)
                    labels.extend(augmented_labels) 
                
                train_sounds.extend(sounds)
                train_labels.extend(labels)
                
#         print(len(train_sounds), len(train_labels))
#         print(len(val_sounds), len(val_labels))
        
        return np.array(train_sounds), np.array(train_labels), np.array(val_sounds), np.array(val_labels)

In [6]:
npz_path = "/workspace/esc/esc10/wav16.npz"
train_sounds, train_labels, val_sounds, val_labels = create_dataset(npz_path,
                                                                        split = 4,
                                                                        fs = 16000, augment_factor = 200)

Preprocessing sounds...
Augmenting data...
Preprocessing sounds...
Augmenting data...
Preprocessing sounds...
Augmenting data...
Preprocessing sounds...
Augmenting data...
Preprocessing sounds...


In [11]:
train_sounds.shape, val_sounds.shape

((64320, 65536), (80, 65536))

In [6]:
# Set size of averaging window T, and signal length N
T = 2**10
N = INPUT_LENGTH

# Set filter bank options
filt_opt_bank = [FiltOptions(Q = 8, T = T,
                          filter_type = 'morlet_1d',
                          boundary = 'symm'),
             FiltOptions(Q = 1, T = T,
                          filter_type = 'morlet_1d',
                          boundary = 'symm')]
# Set scattering options
scat_opt = ScatOptions()
scat_opt.M = 2

# Create filter bank
Wop, bank  = wavelet_factory_1d(N, filt_opt_bank, scat_opt)

In [7]:
def count_scat(x, Wop):
    S, _ = scat(x, Wop)
    
    S_table, _ = format_scat(log_scat(renorm_scat(S)), format_type = 'table')
    print(S_table.shape)
    return S_table    

In [10]:
S_table_train = count_scat(train_sounds.T[:, np.newaxis,:], Wop)
S_table_test = count_scat(val_sounds.T[:, np.newaxis,:], Wop)

MemoryError: 

In [ ]:
esc_save_path = '/workspace/jgusak/Data/esc_saved'

# Uncomment the lines below if you want to save counted coefficients

# np.save(esc_save_path+'esc10_wav16_augfactor9_train_x', S_tables[0])
# np.save(esc_save_path+'esc10_wav16_augfactor9_train_y', labels[0])

# np.save(esc_save_path+'esc10_wav16_val_x', S_tables[1])
# np.save(esc_save_path+'esc10_wav16_val_y', labels[1])

# X_train = S_tables[0]
# y_train = labels[0]

# X_val = S_tables[1]
# y_val = labels[1]